# Preprocessing

In [ ]:
# Uploading dataset to the google drive and mounting drive to access the csv files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

dataset_path = '/content/drive/MyDrive/Facebook-dataset/dataset/'

comments = pd.read_csv(dataset_path + 'comment.csv')
likes = pd.read_csv(dataset_path + 'like.csv')
posts = pd.read_csv(dataset_path + 'post.csv')
members = pd.read_csv(dataset_path + 'member.csv')

# converting all id's to string to avoid further complexities

comments["msg"].values.astype(str)

for column in comments:
  if(column[-2:] == 'id'):
    comments[column] = comments[column].values.astype(str)

for column in likes:
  if(column[-2:] == 'id'):
    likes[column] = likes[column].values.astype(str)

posts["msg"].values.astype(str)

for column in posts:
  if(column[-2:] == 'id'):
    posts[column] = posts[column].values.astype(str)

for column in members:
  if(column[-2:] == 'id'):
    members[column] = members[column].values.astype(str)

In [ ]:
# comment_like_count contains the number of likes a comment has
comment_like_count = dict()

for i in range(len(likes)):
  cid = likes.iloc[i]['cid']
  if cid == 'x':
    # we only want likes for the comments
    continue
  if cid in comment_like_count.keys():
    comment_like_count[cid]+=1
  else:
    comment_like_count[cid] = 1

# comment_post_like contains comment text combined with number of likes that are associated to each post
import math

comment_post_like = dict()

for i in range(len(comments)):
  pid = comments.iloc[i]['pid']
  cid = comments.iloc[i]['cid']
  text = comments.iloc[i]['msg']
  # we don't need the comments of comments
  if comments.iloc[i]['rid'] != "nan":
    continue
  if cid in comment_like_count.keys():
    like_count = comment_like_count[cid]
  if pid in comment_post_like.keys():
    comment_post_like[pid].append((cid,text,like_count))
  else:
    comment_post_like[pid] = [(cid,text,like_count)]

# Keyword Extraction

In [ ]:
# storing the text for each post in the dictionary

post_text = dict()
post_likes = dict()
import numpy

for i in range(len(posts)):
  post_text[posts.iloc[i]["pid"]] = posts.iloc[i]["msg"]
  val = posts.iloc[i]["likes"]
  if math.isnan(val):
    val = numpy.int64(0)
  post_likes[posts.iloc[i]["pid"]] = val

# sorting comments based on the likes they've got

for pid in comment_post_like.keys():
  comment_post_like[pid] = sorted(comment_post_like[pid], key = lambda x : x[2], reverse = True)

# to store the text that represents post (post text + comment text)

post_representation = dict()

# adding post text to the post_representation

for (i,x) in post_text.items():
  post_representation[i] = post_text[i]
  if(type(post_representation[i]) == type(8.9)):
    post_representation[i] = ''

# adding comment text to the post_representation

for pid in comment_post_like.keys():
  if pid not in post_text.keys():
    continue
  for (cid, message, numlikes) in comment_post_like[pid]:
    # empty messages are stored as float in the dataframes
    if(type(message) == type(8.9)):
      continue
    # filtering only the main comments to be used to generate the keywords
    if numlikes >= post_likes[pid]/5 and numlikes > 0:
      post_representation[pid] += message + ' '

In [ ]:
# for regular expressions

import re

def prepreprocess(x):
  res = ''
  bracket_open = False
  btw_text = ''
  for c in x:
    if c == '{':
      bracket_open = True
    elif c == '}' and bracket_open:
      if btw_text == 'COMMA':
        res += ","
      elif btw_text == "APOST":
        res += "'"
      btw_text = ''
      bracket_open = False
    elif not bracket_open:
      res += c
    else:
      btw_text += c

  return re.sub(r'http\S+', '', res)

# pre-preprocessing
# removing {} and links from the text of post_representation

for (i,x) in post_representation.items():
  post_representation[i] = prepreprocess(x)

## TextRank algorithm

In [ ]:
# importing necessary libraries

import nltk
from nltk import word_tokenize
import string
from nltk.stem import WordNetLemmatizer
import numpy as np
import math

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
def clean(text):
  text = text.lower()
  printable = set(string.printable)
  text = list(filter(lambda x: x in printable, text)) #filter funny characters, if any.
  res = ''
  for x in text:
    res += x
  return res

def extract_keywords(Text):
  Cleaned_text = clean(Text)

  res = ''
  
  for x in Cleaned_text:
    if (ord(x) <= ord('z') and ord(x) >= ord('a')) or (ord(x) <= ord('Z') and ord(x) >= ord('A')) or x == "'":
      res += x
    elif x == '.':
      res += x
      res += ' '
    else:
      res += ' '
    
  Cleaned_text = res

  text = word_tokenize(Cleaned_text)

  # POS tagging

  POS_tag = nltk.pos_tag(text)

  # Lemmatization

  wordnet_lemmatizer = WordNetLemmatizer()
  adjective_tags = ['JJ','JJR','JJS']
  lemmatized_text = []
  for word in POS_tag:
    if word[1] in adjective_tags:
      lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0],pos="a")))
    else:
      lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0]))) #default POS = noun
  
  POS_tag = nltk.pos_tag(lemmatized_text)

  # POS based filtering

  stopwords = []

  wanted_POS = ['NN','NNS','NNP','NNPS','JJ','JJR','JJS','VBG','FW'] 

  for word in POS_tag:
    if word[1] not in wanted_POS:
      stopwords.append(word[0])
  
  punctuations = list(str(string.punctuation))

  stopwords = stopwords + punctuations

  # complete stopword generation

  stopword_file = open(dataset_path + "long_stopwords.txt","r")

  lots_of_stopwords = []

  for line in stopword_file.readlines():
    lots_of_stopwords.append(str(line.strip()))
  
  stopwords_plus = []
  stopwords_plus = stopwords + lots_of_stopwords
  stopwords_plus = set(stopwords_plus)

  # removing stopwords

  processed_text = []
  for word in lemmatized_text:
      if word not in stopwords_plus:
          processed_text.append(word)

  # vocabulary creation

  vocabulary = list(set(processed_text))

  # graph creation

  vocab_len = len(vocabulary)

  weighted_edge = np.zeros((vocab_len,vocab_len),dtype=np.float32)

  score = np.zeros((vocab_len),dtype=np.float32)
  window_size = 3
  covered_coocurrences = []

  for i in range(0,vocab_len):
      score[i]=1
      for j in range(0,vocab_len):
          if j==i:
              weighted_edge[i][j]=0
          else:
              for window_start in range(0,(len(processed_text)-window_size+1)):
                  
                  window_end = window_start+window_size
                  
                  window = processed_text[window_start:window_end]
                  
                  if (vocabulary[i] in window) and (vocabulary[j] in window):
                      
                      index_of_i = window_start + window.index(vocabulary[i])
                      index_of_j = window_start + window.index(vocabulary[j])
                      
                      # index_of_x is the absolute position of the xth term in the window 
                      # (counting from 0) 
                      # in the processed_text
                        
                      if [index_of_i,index_of_j] not in covered_coocurrences:
                          weighted_edge[i][j]+=1/math.fabs(index_of_i-index_of_j)
                          covered_coocurrences.append([index_of_i,index_of_j])

  # calculation weighted summation of connections of a vertex

  inout = np.zeros((vocab_len),dtype=np.float32)

  for i in range(0,vocab_len):
      for j in range(0,vocab_len):
          inout[i]+=weighted_edge[i][j]
  
  # scoring vertices

  MAX_ITERATIONS = 10
  d=0.85
  threshold = 0.0001 #convergence threshold

  for iter in range(0,MAX_ITERATIONS):
    prev_score = np.copy(score)
    
    for i in range(0,vocab_len):
      
      summation = 0
      for j in range(0,vocab_len):
        if weighted_edge[i][j] != 0:
          summation += (weighted_edge[i][j]/inout[j])*score[j]
              
      score[i] = (1-d) + d*(summation)
    
    if np.sum(np.fabs(prev_score-score)) <= threshold: #convergence condition
      break
        
  # phrase partitioning

  phrases = []

  phrase = " "
  for word in lemmatized_text:
    
    if word in stopwords_plus:
      if phrase!= " ":
          phrases.append(str(phrase).strip().split())
      phrase = " "
    elif word not in stopwords_plus:
      phrase+=str(word)
      phrase+=" "
    
  phrases = []

  phrase = " "
  for word in lemmatized_text:
    
    if word in stopwords_plus:
      if phrase!= " ":
        phrases.append(str(phrase).strip().split())
      phrase = " "
    elif word not in stopwords_plus:
      phrase+=str(word)
      phrase+=" "
  
  # creating a list of unique phrases

  unique_phrases = []

  for phrase in phrases:
    if phrase not in unique_phrases:
      unique_phrases.append(phrase)
  

  # thinning the list of candidate-keyphrases

  for word in vocabulary:
    #print word
    for phrase in unique_phrases:
      if (word in phrase) and ([word] in unique_phrases) and (len(phrase)>1):
        #if len(phrase)>1 then the current phrase is multi-worded.
        #if the word in vocabulary is present in unique_phrases as a single-word-phrase
        # and at the same time present as a word within a multi-worded phrase,
        # then I will remove the single-word-phrase from the list.
        unique_phrases.remove([word])
  
  # scoring keyphrases

  phrase_scores = []
  keywords = []
  for phrase in unique_phrases:
    phrase_score=0
    keyword = ''
    for word in phrase:
        keyword += str(word)
        keyword += " "
        phrase_score+=score[vocabulary.index(word)]
    phrase_scores.append(phrase_score)
    keywords.append(keyword.strip())

  i=0

  key_scores = dict()

  for keyword in keywords:
    key_scores[keyword] = phrase_scores[i]
    i+=1

  return key_scores


In [ ]:
j = 0

keyword_scores = dict()

for (i,x) in post_representation.items():
  keyword_scores[i] = extract_keywords(x)
  j += 1
  if(j%1000 == 0):
    print("posts completed :",j)

posts completed : 1000
posts completed : 2000
posts completed : 3000
posts completed : 4000
posts completed : 5000
posts completed : 6000
posts completed : 7000
posts completed : 8000
posts completed : 9000
posts completed : 10000
posts completed : 11000
posts completed : 12000
posts completed : 13000
posts completed : 14000
posts completed : 15000
posts completed : 16000
posts completed : 17000
posts completed : 18000
posts completed : 19000
posts completed : 20000


In [ ]:
# for counting the number of words in the given string

def count_words(text):
  result = 0
  cur = ""
  for x in text:
    if x == ' ' and len(cur) > 0:
      result+=1
      cur = ""
      continue
    cur += x
  return result+1

# for getting individual words from the given string/sentances

def get_words(text):
  cur = ""
  res = []
  for x in text:
    if x == ' ':
      if len(cur) > 0:
        res.append(cur)
      cur = ""
    else:
      cur += x
  if len(cur) > 0:
    res.append(cur)
  return res

# for 

def combine(scores):
  for (id, sc) in scores.items():
    cur = dict()
    for (word, score) in sc.items():
      if count_words(word) > 1:
        for x in get_words(word):
          if x in cur.keys():
            cur[x] += score/2
          else:
            cur[x] = score
      else:
        if word in cur.keys():
          cur[word] += score
        else:
          cur[word] = score
    scores[id] = cur
  return scores

textrankscores = combine(keyword_scores)

## TF-IDF ranking based

In [ ]:
# In tf-idf, we are also considering the replies for the first comments (recomments)

post_recomments = dict()

for i in range(len(comments)):
  pid = comments.iloc[i]['pid']
  cid = comments.iloc[i]['cid']
  text = comments.iloc[i]['msg']
  # we only need the comments of comments
  if comments.iloc[i]['rid'] == "nan":
    continue
  if(type(text) == type(9.8)):
    text = ''
  if pid in post_recomments.keys():
    post_recomments[pid] += text
  else:
    post_recomments[pid] = text

# considering the entire text (post text, comments and replies of comments) as a single document

idf_post_representation = dict()

for (i,x) in post_text.items():
  if type(x) == type(8.9):
    idf_post_representation[i] = ''
  else:
    idf_post_representation[i] = x + ' '


# adding comment text to the post_representation

for pid in comment_post_like.keys():
  text = ''
  for (cid, message, numlikes) in comment_post_like[pid]:
    if(type(message) == type(8.9)):
      continue
    text += message
    text += ' '
  if pid not in idf_post_representation.keys():
    idf_post_representation[pid] = text
  else:
    idf_post_representation[pid] += text

for (i,x) in post_recomments.items():
  if i not in idf_post_representation.keys():
    idf_post_representation[i] = x + ' '
  else:
    idf_post_representation[i] += x + ' '

for (i,x) in idf_post_representation.items():
  idf_post_representation[i] = prepreprocess(x)


In [ ]:
# performing the necessary operations for preprocessing the documents (tweet texts) to apply the TF-IDF ranking algorithm

def process_docs(doclist):
  docs = []
  for Text in doclist:
    Cleaned_text = clean(Text)
    res = ''
    
    for x in Cleaned_text:
      if (ord(x) <= ord('z') and ord(x) >= ord('a')) or (ord(x) <= ord('Z') and ord(x) >= ord('A')) or x == "'":
        res += x
      elif x == '.':
        res += x
        res += ' '
      else:
        res += ' '
      
    Cleaned_text = res

    text = word_tokenize(Cleaned_text)

    # POS tagging

    POS_tag = nltk.pos_tag(text)

    # Lemmatization

    wordnet_lemmatizer = WordNetLemmatizer()
    adjective_tags = ['JJ','JJR','JJS']
    lemmatized_text = []
    for word in POS_tag:
      if word[1] in adjective_tags:
        lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0],pos="a")))
      else:
        lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0]))) #default POS = noun
    
    POS_tag = nltk.pos_tag(lemmatized_text)

    # POS based filtering

    stopwords = []

    wanted_POS = ['NN','NNS','NNP','NNPS','JJ','JJR','JJS','VBG','FW'] 

    for word in POS_tag:
      if word[1] not in wanted_POS:
        stopwords.append(word[0])
  
    punctuations = list(str(string.punctuation))

    stopwords = stopwords + punctuations

    # complete stopword generation

    stopword_file = open(dataset_path + "long_stopwords.txt","r")

    lots_of_stopwords = []

    for line in stopword_file.readlines():
      lots_of_stopwords.append(str(line.strip()))
    
    stopwords_plus = []
    stopwords_plus = lots_of_stopwords
    stopwords_plus = set(stopwords_plus)

    # removing stopwords

    processed_text = []
    for word in lemmatized_text:
      if word not in lots_of_stopwords:
        processed_text.append(word)

    res = ""
    for x in processed_text:
      if len(res) > 0:
        res += " "
      res += x
    docs.append(res)
  
  return docs

  

In [ ]:
def pre_process(text):
  text = text.lower()
  text = re.sub("&lt;/?.*&gt;","&lt;&gt; ", text)
  text = re.sub("(\\d|\\W)+"," ",text)
  return text

for (i,x) in idf_post_representation.items():
  idf_post_representation[i] = pre_process(x)

from sklearn.feature_extraction.text import CountVectorizer

def get_stop_words(stop_file_path):
  with open(stop_file_path,'r', encoding = 'utf-8') as f:
    stopwords = f.readlines()
    stop_set = set(m.strip() for m in stopwords)
    return stop_set

stopwords = get_stop_words(dataset_path + "stopwords.txt")

docs = []

for (i,x) in idf_post_representation.items():
  docs.append(x)


In [ ]:
docs = process_docs(docs)

In [ ]:
# from sklearn.feature_extraction import text 
# stop_words = text.ENGLISH_STOP_WORDS.union(stopwords)

In [ ]:
print(type(list(stopwords)))
# print(type(docs[0]))

<class 'list'>


In [ ]:
cv = CountVectorizer(max_df=0.85,stop_words = list(stopwords))
word_count_vector = cv.fit_transform(docs)

from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer(smooth_idf = True, use_idf = True)
tfidf_transformer.fit(word_count_vector)

feature_names = cv.get_feature_names_out()

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['come', 'vis', 'viser', 'visest'] not in stop_words.
  warnings.warn(


In [ ]:
def extract(coo_matrix):
  tuples = zip(coo_matrix.col, coo_matrix.data)
  score_vals = []
  feature_vals = []

  for idx, score in sorted(tuples):
    score_vals.append(score)
    feature_vals.append(feature_names[idx])
  
  result = dict()

  for idx in range(len(feature_vals)):
    result[feature_vals[idx]] = score_vals[idx]
  
  return result


In [ ]:
def map_scale1_convert(scores):
  mx = 0
  for (word, score) in scores.items():
    mx = max(mx, score)
  for (word, score) in scores.items():
    scores[word] = score/mx
  return scores

In [ ]:
import math

keywords = dict()

postcount = 0

Count = 0

for pid in post_text.keys():
  postcount += 1
  if postcount % 1000 == 0:
    print('posts completed :', postcount)
  text = post_text[pid]
  if type(text) == type(8.9):
    text = ''
  text = prepreprocess(text)
  text = process_docs([text])
  text = text[0]
  post_scores = extract(tfidf_transformer.transform(cv.transform([text])).tocoo())

  if pid in comment_post_like.keys():
    num_comments = 0
    total_likes = 0
    for (cid, message, numlikes) in comment_post_like[pid]:
      num_comments += 1
      total_likes += numlikes

    multiplier = dict()
    ctext = ''
    for (cid, message, numlikes) in comment_post_like[pid]:
      if(type(message) == type(8.9)):
        continue
      ctext += prepreprocess(message) + ' '

      comment_scores = extract(tfidf_transformer.transform(cv.transform([prepreprocess(message)])).tocoo())

      for (word, score) in comment_scores.items():
        if word in multiplier.keys():
          multiplier[word] += (1 + numlikes)
        else:
          multiplier[word] = (1 + numlikes)

  comment_scores = extract(tfidf_transformer.transform(cv.transform([ctext])).tocoo())
  for (word, score) in comment_scores.items():
    if word in post_scores.keys():
      post_scores[word] += max(multiplier[word]/(num_comments+total_likes), 1/2)*score
    else:
      post_scores[word] = max(multiplier[word]/(num_comments+total_likes), 1/3)*score

  recomment_scores = dict()
  if pid in recomment_scores.keys():
    recomment_scores = extract(tfidf_transformer.transform(cv.transform([prepreprocess(post_recomments[pid])])).tocoo())

  for (word, score) in recomment_scores.items():
    if word in post_scores.keys():
      post_scores[word] += 1/3*score
    else:
      post_scores[word] = 1/10*score
  
  post_scores = map_scale1_convert(post_scores)

  textrankscores[pid] = map_scale1_convert(textrankscores[pid])

  for (word, score) in textrankscores[pid].items():
    if word in post_scores.keys():
      post_scores[word] += score
    else:
      post_scores[word] = score

  post_scores = dict(sorted(post_scores.items(), key=lambda item: item[1], reverse = True))

  num_keywords = max(5, count_words(text)/4)
  num_keywords = min(25, num_keywords)

  words = []

  count = 0
  for (word, scores) in post_scores.items():
    if(count >= num_keywords):
      break
    count += 1
    words.append(word)
      
  keywords[pid] = words


posts completed : 1000
posts completed : 2000
posts completed : 3000
posts completed : 4000
posts completed : 5000
posts completed : 6000
posts completed : 7000
posts completed : 8000
posts completed : 9000
posts completed : 10000
posts completed : 11000
posts completed : 12000
posts completed : 13000
posts completed : 14000
posts completed : 15000
posts completed : 16000
posts completed : 17000
posts completed : 18000
posts completed : 19000
posts completed : 20000


# Tag network and kernel diffusion

In [ ]:
def remove_singles(keywords):
  word_count = dict()
  for (id, kwords) in keywords.items():
    for x in kwords:
      if x in word_count.keys():
        word_count[x] += 1
      else:
        word_count[x] = 1
  res = dict()
  for (id, kwords) in keywords.items():
    newwords = []
    for x in kwords:
      if word_count[x] > 1:
        newwords.append(x)
    res[id] = newwords
  return res

In [ ]:
# removing multiple worded keywords and combining them into one

keywords = remove_singles(keywords)

# maintaining number to represent the tags instead of the strings
tags = []
tagHash = dict()

for (id, kwords) in keywords.items():
  for x in kwords:
    if x in tagHash.keys():
      continue
    tagHash[x] = len(tags)
    tags.append(x)

In [ ]:
# userposts is a dictionary mapping userids to the list of posts of the user

userposts = dict()

# iterating through the posts DataFrame and populating the userposts datastructure

for i in range(len(posts)):
  userid = posts.iloc[i]['id']
  if userid in userposts.keys():
    userposts[userid].append(posts.iloc[i]['pid'])
  else:
    userposts[userid] = [posts.iloc[i]['pid']]

# it's easier to maintain strings in tags in form of numbers than the strings themselves
# so replacing the tags in the keywords datastructure with their respective hash values

for (id, kwords) in keywords.items():
  hashed = []
  for x in kwords:
    hashed.append(tagHash[x])
  keywords[id] = hashed

num_tags = len(tags)

# creating tag network 

tag_network = np.zeros([num_tags, num_tags])

# iterating through all the posts posted by the user and populating the tag_network giving the appropriate weights
# note: the weight of tag_network[x][y] is number of users who have used both the tags x and y simultaneously

for (id, posts) in userposts.items():
  # howmany times a user might use the same pair of keywords simultaneously, we only consider it once
  paircount = dict()
  for pid in posts:
    # for every post iterating through all the pairwise tags
    if pid not in keywords.keys():
      continue
    for i in range(len(keywords[pid])):
      for j in range(i+1):
        x = keywords[pid][i]
        y = keywords[pid][j]
        if x > y:
          x,y = y,x
        if (x,y) in paircount.keys():
          paircount[(x,y)] += 1
        else:
          paircount[(x,y)] = 1
  for (x,y) in paircount.keys():
    tag_network[x][y] += 1
    tag_network[y][x] += 1

# representing a user with number between 0 - num_users rather than the long user_id  
userHash = dict()
# contains all the userids
userids = []

for id in userposts.keys():
  if id in userHash.keys():
    continue
  userHash[id] = len(userids)
  userids.append(id)

num_users = len(userids)


newuserposts = dict()

for (id, posts) in userposts.items():
  newuserposts[userHash[id]] = posts

userposts = newuserposts

usertagweights = np.zeros([num_users, num_tags])

for (id, posts) in userposts.items():
  for pid in posts:
    if pid not in keywords.keys():
      continue
    for tag in keywords[pid]:
      usertagweights[id][tag] += 1

In [ ]:
# Has some functionalities that are required to calculate the kernel
import scipy.linalg


# this class is to find the exponential of the given matrix (also known as kernel exponentiation)
# note: only works for symmetric matrices

class kernelexponentiation:
  def __init__(self, mat):

    self.n = len(mat)

    # column sum is an array storing the sum of elements in each column

    column_sum = numpy.zeros(self.n)
    for i in range(self.n):
      for j in range(self.n):
        column_sum[j] += mat[i][j]

    D = numpy.diag(column_sum)

    # finding the negated leplacian matrix which helps in calculatint the exponential of a matrix

    self.mat = numpy.subtract(mat, D)

    # self.EV contains the eigen values of calculated negated leplacians matrix
    # similarly self.V contains the eigen vectors and self.VT is the transpose of eigen vectors 
    curres = numpy.linalg.eig(self.mat)
    self.EV = curres[0]
    self.V = curres[1]
    self.VT = numpy.transpose(self.V)

  # computing the exponential using the formuala
  def get_kernel(self,beta):
    res = numpy.matmul(self.V, scipy.linalg.expm(numpy.diag(beta*self.EV)))
    res = numpy.matmul(res, self.VT)
    for i in range(num_tags):
      for j in range(num_tags):
        # removing the complex values
        res[i][j] = res[i][j].real
    return res

# takes range [l,r] and returns n unique numbers from the range randomly

def get_rand(n, l, r):
  res = []
  while len(res) < n:
    cur = numpy.random.randint(l,r)
    if cur in res:
      continue
    res.append(cur)
  return res

In [ ]:
# the class mentioned above is loaded into the variable kexp with the matrix being tag_network

kexp = kernelexponentiation(tag_network)

In [ ]:
K = kexp.get_kernel(0.00001)

In [ ]:
distances_matrix = np.zeros([num_users, num_users])

# nonzerotags stores only the tags that are used by the user for each user
nonzerotags = []
denoms = []

for i in range(num_users):
  cur = []
  denom = 0
  for j in range(num_tags):
    # considering if only weidht is greater than 0
    if usertagweights[i][j] > 0:
      # precalculating the denominators
      denom += usertagweights[i][j]*usertagweights[i][j]
      cur.append(j)
  nonzerotags.append(cur)
  denom **= 0.5
  denoms.append(denom)

for i in range(num_users):
  for j in range(i):
    for x in nonzerotags[i]:
      for y in nonzerotags[j]:
        # using kernel to find the similarity between the users
        distances_matrix[i][j] += K[x][y]*usertagweights[i][x]*usertagweights[j][y]
    distances_matrix[i][j] /= denoms[i]*denoms[j]
    distances_matrix[j][i] = distances_matrix[i][j]

<ipython-input-32-231ac3220a26>:25: ComplexWarning: Casting complex values to real discards the imaginary part
  distances_matrix[i][j] += K[x][y]*usertagweights[i][x]*usertagweights[j][y]
<ipython-input-32-231ac3220a26>:26: RuntimeWarning: invalid value encountered in double_scalars
  distances_matrix[i][j] /= denoms[i]*denoms[j]


In [ ]:
import csv

# saving the value of beta calculated to this point into a csv file
# the computational cost of calculating the kernel (K) is expensive and we cannot afford to compute the value of kernel everytime we run the notebook

with open('drive/MyDrive/userdistances.csv', 'w', encoding='UTF8', newline='') as f:
  f.truncate()
  writer = csv.writer(f)
  dummy = [x for x in range(num_users)]
  writer.writerow(dummy)
  writer.writerows(distances_matrix)


# Distances matrix saved, can run from here

In [ ]:
# For loading the similarity matrix from the drive

data = pd.read_csv("drive/MyDrive/userdistances.csv")
distances_matrix = data.to_numpy()

num_users = len(distances_matrix)

# Agglomerative Hierarchical clustering

In [ ]:
# normalizes a 2d matrix from 0 to 1

def normalize(mat):
  mx = mat[0][0]
  mn = mat[0][0]

  for i in range(len(mat)):
    for j in range(len(mat[i])):
      mx = max(mat[i][j], mx)
      mn = min(mat[i][j], mn)
  
  for i in range(len(mat)):
    for j in range(len(mat[i])):
      mat[i][j] = (mat[i][j]-mn)/(mx-mn)
  
  return mat

In [ ]:

from matplotlib import pylab

# For plotting the graphs
import matplotlib.pyplot as plt

# takes the graph and saves the visualization into google drive

def save_graph(graph,file_name, num_nodes, main_nodes = [], sizes = None):

  # sizes of all the nodes is 200 if not specified
  if sizes == None:
    sizes = []
    for x in range(num_nodes):
      sizes.append(200)
  plt.figure(num=None, figsize=(60, 60), dpi=40)
  plt.axis('off')
  fig = plt.figure(1)

  # this layout avoids overlapping of clusters
  pos = graphviz_layout(graph, prog = 'neato')
  nx.draw_networkx_nodes(graph,pos, node_size = sizes)
  nx.draw_networkx_edges(graph,pos, width = 1)
  nx.draw_networkx_labels(graph,pos)

  cut = 1.00
  xmax = cut * max(xx for xx, yy in pos.values())
  ymax = cut * max(yy for xx, yy in pos.values())
  plt.xlim(0, xmax + 20)
  plt.ylim(0, ymax + 20)

  plt.savefig(file_name,bbox_inches="tight")
  pylab.close()
  del fig

In [ ]:
# Heap datastructure has a major in the agglomerative clustering algorithm that has been used 
import heapq

# For copying arrays and to avoid copying the addresses instead
from copy import deepcopy


# For clustering visualization
import networkx as nx

import pydot
from networkx.drawing.nx_pydot import graphviz_layout


# this class peforms the agglomerative clustering algorithm

class agglomerative_clustering:

  # setting all the appropriate values

  def __init__(self, distance_matrix):
    # self.dist stores the distance_matrices    
    self.dist = distance_matrix
    self.num_nodes = len(self.dist)
    # performing normalization
    self.dist = normalize(self.dist)
    # self.num_clusters indicates the current number of clusters
    self.num_clusters = self.num_nodes
    # self.inter_cluster_distances is a 2d matrix that stores the summation of the distances among the nodes in the two clusters
    self.inter_cluster_distances = deepcopy(self.dist)
    # self.is_cluster_head indicates whether a node is head of its cluster
    self.is_cluster_head = [True for x in range(self.num_nodes)]
    # self.belongs_to stores the head node of the cluster which the current node belongs to
    self.belongs_to = [x for x in range(self.num_nodes)]
    # self.cluster[head_node] stores the nodes in the clusters whose head node is head_node 
    self.cluster = [[x] for x in range(self.num_nodes)]
    # self.merges records all the merges that are done
    self.merges = []
    # self.heap stores the inter_cluster_distances and dynamically returns the closest clusters each time
    # each element of the heap is in format [distances, head node of cluster 1, head node of cluster 2, iteration when pushed]
    # note: the iteration of pushing is noted to identify whether to identify the distances after the push has happened
    self.heap = []
    # self.last_updated stores the last iteration in which the inter_cluster_distances between the two clusters was updated
    self.last_updated = np.zeros([self.num_nodes, self.num_nodes])
    # self.iter stores the current number of iterations performed
    self.iter = 0

    
  def visualize(self):
    # color values on the scale from 0 to 1 on 'Set2' color map
    sizes = []
    G=nx.Graph()
    for i in range(self.num_nodes):
      G.add_node(i)
      # head nodes to be bigger in sizes to be identifiable
      if self.is_cluster_head[i] and len(self.cluster[i]) > 4:
        sizes.append(1000)
      else:
        sizes.append(300)

    # adding edges to the graph
    for edge in self.merges:
      G.add_edge(edge[0],edge[1])

    # specifying path for graph that needs to be saved"
    path = "drive/MyDrive/UsersClusteringVisualizationGraph-" + str(self.num_clusters) + ".pdf"

    save_graph(G,path, self.num_nodes, sizes = sizes)

  # for getting the average distance between nodes in two clusters from inter_cluster_distances by dividing with appropriate value
  def get_inter_cluster_distance(self, u, v):
    return self.inter_cluster_distances[u][v]/(len(self.cluster[u])*len(self.cluster[v]))

  # this is the point where clustering starts
  def go(self, visualization = False, max_cluster_size = 10**10):
    
    # initially populating the heap

    for i in range(self.num_nodes):
      for j in range(self.num_nodes):
        if i >= j:
          continue
        self.heap.append([-self.dist[i][j], i, j, self.iter])
    # getting the top element heap (and finding the most similar clusters)
    heapq.heapify(self.heap)
    # performing the clustering untill all the nodes are merged
    while len(self.heap) != 0:
      # u represents to top element in the heap
      u = heapq.heappop(self.heap)
      if self.is_cluster_head[u[1]] and self.is_cluster_head[u[2]] and self.last_updated[u[1]][u[2]] == u[3] and (len(self.cluster[u[1]]) + len(self.cluster[u[2]]) <= max_cluster_size):
        # incrementing the number of iterations
        self.iter += 1
        # combining the two clusters using the funtion that is defined below
        self.combine(u[1],u[2])
        self.num_clusters-=1
        # printing the scores
        if self.num_clusters%500 == 0 or (self.num_clusters <= 500 and self.num_clusters%100 == 0) or (self.num_clusters <= 100 and self.num_clusters%10 == 0) or (self.num_clusters <= 10):
          if(visualization):
            self.visualize()

  # this function takes head nodes and combines the clusters corresponding to the head nodes
  def combine(self, x, y):
    # recording the merge
    self.merges.append([x,y])
    # new head of the cluster
    # note: new head is always the head node of the largest cluster as the cluster with smaller size is merged into the cluster with larger size
    new_head = x
    # pilla_cluster_head represents the head of the smaller cluster
    pilla_cluster_head = y
    if len(self.cluster[x]) < len(self.cluster[y]):
      new_head = y
      pilla_cluster_head = x
    
    # changing the status of is_cluster_head for the head of smaller cluster as it is no longer head of the cluster
    self.is_cluster_head[pilla_cluster_head] = False

    # recalculatint the inter_cluster_distances for all clusters to the current cluster as new nodes are added into the cluster
    for i in range(self.num_nodes):
      if self.is_cluster_head[i] == False or (i == new_head):
        continue
      
      self.inter_cluster_distances[i][new_head] += self.inter_cluster_distances[i][pilla_cluster_head]
      self.inter_cluster_distances[new_head][i] = self.inter_cluster_distances[i][new_head]
      # recording the latest updation
      self.last_updated[i][new_head] = self.iter
      self.last_updated[new_head][i] = self.iter
      denom = (len(self.cluster[i])*(len(self.cluster[new_head]) + len(self.cluster[pilla_cluster_head])))
      # pushing the updated value
      heapq.heappush(self.heap, [-self.inter_cluster_distances[i][new_head]/denom, i, new_head, self.iter])

    # merging the nodes into the new cluster
    for node in self.cluster[pilla_cluster_head]:
      self.belongs_to[node] = new_head
      self.cluster[new_head].append(node)


In [ ]:
# performing clustering using the class declared above
clustering_object = agglomerative_clustering(distances_matrix)
# note: clusters are visualized if only specified
clustering_object.go(visualization=True, max_cluster_size = 100)

<ipython-input-36-25f4483a08d9>:20: DeprecationWarning: nx.nx_pydot.graphviz_layout depends on the pydot package, which hasknown issues and is not actively maintained. Consider usingnx.nx_agraph.graphviz_layout instead.

See https://github.com/networkx/networkx/issues/5723
  pos = graphviz_layout(graph, prog = 'neato')
<ipython-input-36-25f4483a08d9>:20: DeprecationWarning: nx.nx_pydot.graphviz_layout depends on the pydot package, which hasknown issues and is not actively maintained. Consider usingnx.nx_agraph.graphviz_layout instead.

See https://github.com/networkx/networkx/issues/5723
  pos = graphviz_layout(graph, prog = 'neato')
<ipython-input-36-25f4483a08d9>:20: DeprecationWarning: nx.nx_pydot.graphviz_layout depends on the pydot package, which hasknown issues and is not actively maintained. Consider usingnx.nx_agraph.graphviz_layout instead.

See https://github.com/networkx/networkx/issues/5723
  pos = graphviz_layout(graph, prog = 'neato')
<ipython-input-36-25f4483a08d9>:20: D